In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [2]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_shenghuo_gaoxiao=pd.DataFrame(columns=columns)
channel_shenghuo_gaoxiao

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [16]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [17]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [5]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[9]/div/a/span').click()  #选择“生活”频道

In [18]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[9]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'生活'

In [19]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[2]/a').click()  #选择“搞笑”子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[2]/a').text
sub_channel

'搞笑'

In [21]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [24]:
print(datetime.datetime.now())

for month in range(7,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, 700)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_shenghuo_gaoxiao.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 

2020-12-22 07:41:28.799174


<ipython-input-24-7c37149749f2>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-24-7c37149749f2>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-24-7c37149749f2>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为145411a7wd视频属性的抓取
BV号为145411a7wd的视频属性抓取结束


<ipython-input-24-7c37149749f2>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-24-7c37149749f2>:187: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1Rt4y197zN视频属性的抓取
BV号为1Rt4y197zN的视频属性抓取结束
开始BV号为13f4y1R7vX视频属性的抓取
BV号为13f4y1R7vX的视频属性抓取结束
开始BV号为1n5411Y7jR视频属性的抓取
BV号为1n5411Y7jR的视频属性抓取结束
开始BV号为15f4y1R7L2视频属性的抓取
BV号为15f4y1R7L2的视频属性抓取结束
开始BV号为19T4y177qD视频属性的抓取
BV号为19T4y177qD的视频属性抓取结束
开始BV号为1aD4y1S7mX视频属性的抓取
BV号为1aD4y1S7mX的视频属性抓取结束
开始BV号为1HT4y1E7wm视频属性的抓取
BV号为1HT4y1E7wm的视频属性抓取结束
开始BV号为17K411H7Lb视频属性的抓取
BV号为17K411H7Lb的视频属性抓取结束
开始BV号为1xK411J7xZ视频属性的抓取
BV号为1xK411J7xZ的视频属性抓取结束
开始BV号为1yp4y1i7be视频属性的抓取
开始BV号为1yp4y1i7be视频属性的抓取
BV号为1yp4y1i7be的视频属性抓取结束
开始BV号为1mv411q7YG视频属性的抓取
BV号为1mv411q7YG的视频属性抓取结束
开始BV号为1Dh411o7bQ视频属性的抓取
BV号为1Dh411o7bQ的视频属性抓取结束
开始BV号为1AT4y1E7fn视频属性的抓取
BV号为1AT4y1E7fn的视频属性抓取结束
开始BV号为1Gf4y1R7Tv视频属性的抓取
BV号为1Gf4y1R7Tv的视频属性抓取结束
开始BV号为16D4y1S7fz视频属性的抓取
BV号为16D4y1S7fz的视频属性抓取结束
开始BV号为14T4y1j7Jj视频属性的抓取
BV号为14T4y1j7Jj的视频属性抓取结束
开始BV号为1ti4y1V7xA视频属性的抓取
BV号为1ti4y1V7xA的视频属性抓取结束
开始BV号为19i4y137RJ视频属性的抓取
BV号为19i4y137RJ的视频属性抓取结束
开始BV号为1gK4y1v7bF视频属性的抓取
BV号为1gK4y1v7bF的视频属性抓取结束
开始BV号为1YT4y1E7TA视频属性的抓取
BV号为1YT4y1E7TA的视频属性抓取结束
开始BV号为1jD4y1U7vV

BV号为1Q54y1m7Ga的视频属性抓取结束
开始BV号为1KK4y1e7Ve视频属性的抓取
BV号为1KK4y1e7Ve的视频属性抓取结束
开始BV号为1Qy4y1C72M视频属性的抓取
BV号为1Qy4y1C72M的视频属性抓取结束
开始BV号为1kf4y197vP视频属性的抓取
BV号为1kf4y197vP的视频属性抓取结束
开始BV号为1xk4y127D5视频属性的抓取
BV号为1xk4y127D5的视频属性抓取结束
开始BV号为1PK4y1e7eh视频属性的抓取
BV号为1PK4y1e7eh的视频属性抓取结束
开始BV号为1rA411n7Z6视频属性的抓取
BV号为1rA411n7Z6的视频属性抓取结束
开始BV号为1wC4y1b79H视频属性的抓取
BV号为1wC4y1b79H的视频属性抓取结束
开始BV号为1K5411b7hg视频属性的抓取
BV号为1K5411b7hg的视频属性抓取结束
开始BV号为1i541187ja视频属性的抓取
BV号为1i541187ja的视频属性抓取结束
开始BV号为1E54y1i7qW视频属性的抓取
BV号为1E54y1i7qW的视频属性抓取结束
开始BV号为12K4y1Y7ko视频属性的抓取
BV号为12K4y1Y7ko的视频属性抓取结束
开始BV号为1Ni4y1g7fu视频属性的抓取
BV号为1Ni4y1g7fu的视频属性抓取结束
开始BV号为1jZ4y1M7EX视频属性的抓取
BV号为1jZ4y1M7EX的视频属性抓取结束
开始BV号为1iA411Y7fA视频属性的抓取
BV号为1iA411Y7fA的视频属性抓取结束
开始BV号为1TA411Y7yE视频属性的抓取
BV号为1TA411Y7yE的视频属性抓取结束
开始BV号为1NC4y1t7sx视频属性的抓取
BV号为1NC4y1t7sx的视频属性抓取结束
开始BV号为1q54y1i7hB视频属性的抓取
BV号为1q54y1i7hB的视频属性抓取结束
开始BV号为1RC4y1t7a1视频属性的抓取
BV号为1RC4y1t7a1的视频属性抓取结束
开始BV号为1bv411v7Hg视频属性的抓取
BV号为1bv411v7Hg的视频属性抓取结束
开始BV号为1kp4y1Y7T7视频属性的抓取
BV号为1kp4y1Y7T7的视频属性抓取结束
开始BV号为1Ft4y1D7jy

开始BV号为1JZ4y1L7iX视频属性的抓取
BV号为1JZ4y1L7iX的视频属性抓取结束
开始BV号为1yA411L7W9视频属性的抓取
BV号为1yA411L7W9的视频属性抓取结束
开始BV号为1yD4y1d7cA视频属性的抓取
BV号为1yD4y1d7cA的视频属性抓取结束
开始BV号为1Ur4y1c75J视频属性的抓取
BV号为1Ur4y1c75J的视频属性抓取结束
开始BV号为1uy4y1r7Qh视频属性的抓取
BV号为1uy4y1r7Qh的视频属性抓取结束
开始BV号为1Fk4y1y7vM视频属性的抓取
BV号为1Fk4y1y7vM的视频属性抓取结束
开始BV号为17f4y1B7fq视频属性的抓取
BV号为17f4y1B7fq的视频属性抓取结束
开始BV号为1st4y1v7vj视频属性的抓取
BV号为1st4y1v7vj的视频属性抓取结束
开始BV号为1g541177tk视频属性的抓取
BV号为1g541177tk的视频属性抓取结束
开始BV号为16y4y1r7Nd视频属性的抓取
BV号为16y4y1r7Nd的视频属性抓取结束
开始BV号为1aa411A7VC视频属性的抓取
BV号为1aa411A7VC的视频属性抓取结束
开始BV号为1Rf4y1B7Yq视频属性的抓取
BV号为1Rf4y1B7Yq的视频属性抓取结束
开始BV号为13v411y7jY视频属性的抓取
BV号为13v411y7jY的视频属性抓取结束
开始BV号为1Nv411k7hk视频属性的抓取
BV号为1Nv411k7hk的视频属性抓取结束
开始BV号为11T4y1c79t视频属性的抓取
BV号为11T4y1c79t的视频属性抓取结束
开始BV号为1Mv411k7Cv视频属性的抓取
BV号为1Mv411k7Cv的视频属性抓取结束
开始BV号为1Vp4y1k7Ua视频属性的抓取
BV号为1Vp4y1k7Ua的视频属性抓取结束
开始BV号为1Ga411A7k5视频属性的抓取
BV号为1Ga411A7k5的视频属性抓取结束
开始BV号为1hT4y1c7Cc视频属性的抓取
BV号为1hT4y1c7Cc的视频属性抓取结束
开始BV号为1n54y1k7cG视频属性的抓取
BV号为1n54y1k7cG的视频属性抓取结束
开始BV号为1fk4y1C7Hf视频属性的抓取
BV号为1fk4y1C7Hf的视

In [14]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-14-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [25]:
channel_shenghuo_gaoxiao.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97
1月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
3月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
4月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
5月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
6月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
7月,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98
8月,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96


In [26]:
channel_shenghuo_gaoxiao.to_json('channel_shenghuo_gaoxiao.json')